# Launch and set up a VM instance- with python-chi

In [1]:
from chi import server, context
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

In [3]:
username = os.getenv('USER') # all exp resources will have this prefix
server_name = f"project-group7-node-persist"
s = server.Server(
    f"project-group7-node-persist", 
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.large"
)
s.submit(idempotent=True)

Waiting for server project-group7-node-persist's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,project-group7-node-persist
Id,086adb42-04c5-4d54-beb5-d9fc2e3ad14b
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.large
Addresses,sharednet1: IP: 10.56.2.148 (v4) Type: fixed MAC: fa:16:3e:45:0e:09
Network Name,sharednet1
Created At,2025-05-05T19:45:56Z
Keypair,jt4850_nyu_edu-jupyter
Reservation Id,None
Host Id,975ebaa60f3d2240a16bf0f4949a97deee0b14198391845c37f29b19


In [4]:
s.associate_floating_ip()

In [5]:
reserved_fip = s.get_floating_ip()
print(reserved_fip)

129.114.27.198


In [6]:
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by MLFlow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"}
]

In [7]:
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:

  if not os_conn.get_security_group(sg['name']):
      os_conn.create_security_group(sg['name'], sg['description'])
      os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp', remote_ip_prefix='0.0.0.0/0')

  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

updated security groups: ['allow-8000', 'allow-8888', 'allow-9000', 'allow-9001', 'allow-ssh', 'default']


In [8]:
s.refresh()
s.check_connectivity()

Checking connectivity to 129.114.27.198 port 22.


Connection successful


In [9]:
# set up docker
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.27.198: b'b208ddebabeb5daf6a2f2cd3e1d01039'
  warnings.warn(


# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null

Running kernel seems to be up-to-date.

Restarting services...
 systemctl restart packagekit.service

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

Now we want do clone the repo from github

then, we will set up the container framwork

now we can ssh into our server. where abcd is the FIP you get from above

In [ ]:
ssh -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D


Open the GUI for CHI@TACC:

from the Chameleon website
click “Experiment” > “CHI@TACC”
log in if prompted to do so
check the project drop-down menu near the top left (which shows e.g. “CHI-XXXXXX”), and make sure the correct project is selected.
In the menu sidebar on the left side, click on “Object Store” > “Containers” and then, “Create Container”. You will be prompted to set up your container step by step using a graphical “wizard”.

Specify the name as whatever you like.
Leave other settings at their defaults, and click “Submit”.

In the menu sidebar on the left side of the Horizon GUI, click “Identity” > “Application Credentials”. Then, click “Create Application Credential”.

In the “Name”, field, use “data-persist”.
Set the “Expiration” date when you want it expiration.
Click “Create Application Credential”.
Copy the “ID” and “Secret” displayed in the dialog, and save them in a safe place. You will not be able to view the secret again from the Horizon GUI. Then, click “Download openrc file” to have another copy of the secret.

then run following code on node -persist

curl https://rclone.org/install.sh | sudo bash  
sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf  
mkdir -p ~/.config/rclone  
nano  ~/.config/rclone/rclone.conf

modify the file with your own identity, cause this is a public repo, so I dont post mine here

[chi_tacc]  
type = swift  
user_id = YOUR_USER_ID  
application_credential_id = APP_CRED_ID  
application_credential_secret = APP_CRED_SECRET  
auth = https://chi.tacc.chameleoncloud.org:5000/v3  
region = CHI@TACC

# creat a pipeline this part is also the Data pipeline part

you can view the code here https://github.com/Ruibin2000/MLSystemProject_rc5018_jt4850_sj4025/blob/main/dapa_pipeline/docker-compose-etl

we need to clone our repo into our machine

git clone https://github.com/Ruibin2000/MLSystemProject_rc5018_jt4850_sj4025.git


cd MLSystemProject_rc5018_jt4850_sj4025/dapa_pipeline


after you in this dir, we can just run  
curl -L "https://zenodo.org/records/14935094/files/plantsegv3.zip?download=1" -o plantsegv3.zip

then run 
docker compose -f ~/data-persist-chi/docker/docker-compose-etl.yaml run extract-data  
export RCLONE_CONTAINER=object-persist-netID
docker compose -f ~/data-persist-chi/docker/docker-compose-etl.yaml run load-data

# mount an object

in terminal run  
sudo mkdir -p /mnt/object  
sudo chown -R cc /mnt/object  
sudo chgrp -R cc /mnt/object  
rclone mount chi_tacc:you-name-choosed /mnt/object --read-only --allow-other --daemon


ls /mnt/object


# Using block storage

Then we need to creat volume in kvm@tacc  
click on “Volumes” > “Volumes” and then, “Create Volume”. You will be prompted to set up your volume step by step using a graphical “wizard”.   

Specify the name  
Specify the size as 3gb  
click “Create Volume”.


From “Volumes” > “Volumes”, next to your volume, click the ▼ in the menu on the right and choose “Manage Attachments”. In the “Attach to Instance” menu, choose your compute instance. Then, click “Attach Volume”.

run in terminal to check if it working

lsblk 

then run 
sudo parted -s /dev/vdb mklabel gpt  
sudo parted -s /dev/vdb mkpart primary ext4 0% 100%    
sudo mkdir -p /mnt/block  
sudo mount /dev/vdb1 /mnt/block  
sudo chown -R cc /mnt/block  
sudo chgrp -R cc /mnt/block  
df -h

the output of df-h should look like this  
Filesystem                    Size  Used Avail Use% Mounted on  
tmpfs                         795M  1.6M  793M   1% /run  
/dev/vda3                      37G   16G   20G  45% /  
tmpfs                         3.9G     0  3.9G   0% /dev/shm  
tmpfs                         5.0M     0  5.0M   0% /run/lock  
/dev/vda1                     549M  184K  549M   1% /boot/efi  
tmpfs                         795M   12K  795M   1% /run/user/1000  
/dev/vdb1                     4.9G  4.5G  138M  98% /mnt/block  
chi_uc:object-persist-group7  1.1P  1.7G  1.0P   1% /mnt/object-uc  
chi_tacc:mlops_group7         1.1P  1.7G  1.0P   1% /mnt/object  


and the output of lsblk should be looks like this  
NAME   MAJ:MIN RM  SIZE RO TYPE MOUNTPOINTS  
vda    253:0    0   40G  0 disk   
├─vda1 253:1    0  550M  0 part /boot/efi  
├─vda2 253:2    0    8M  0 part   
└─vda3 253:3    0 39.5G  0 part /  
vdb    253:16   0    5G  0 disk   
└─vdb1 253:17   0    5G  0 part /mnt/block  

the output of ls /mnt/object should have your data in it, and you should be able to download your data from horizon gui also  
cc@project-group7-node-persist:~$ ls /mnt/object  
annotations  annotation_test.json  annotation_train.json  annotation_val.json  images  Metadatav2.csv

then we will make the persistent storage helpful,you can run

HOST_IP=$(curl --silent http://169.254.169.254/latest/meta-data/public-ipv4 ) docker compose -f docker-compose-block.yaml up -d


which will give you mlflow , Postgresql database, MinIO object and a Jupyter server

you can access mlfow by http://A.B.C.D:8000
where abcd is your floating ip

just put this in your training python code, then you can have access of mlflow.

Then if your friend use your mlflow service, you will see file change in /mnt/block, example output blow.

cc@project-group7-node-persist:~$ ls /mnt/block/minio_data  

mlflow-artifacts  

cc@project-group7-node-persist:~$ ls /mnt/block/minio_data/mlflow-artifacts  
1  2  